In [ ]:
import pandas as pd
import numpy as np
from SALib import ProblemSpec
from SALib.sample import saltelli
import os

# Set random seed
np.random.seed(42)

n_samples = 50000
n_params = 22

# Directory to save results
output_dir = "Sensitivity_Analysis_Results/Simulator"
os.makedirs(output_dir, exist_ok=True)


In [ ]:
# Load input and output data
df_x = pd.read_csv(f'../Emulation/Input/input_{n_samples}_{n_params}params.csv')
boolean_index = pd.read_csv(f"../Emulation/Outputs/Output_{n_samples}_{n_params}params/bool_indices_{n_samples}.csv")
output = pd.read_csv(f'../Emulation/Outputs/Output_{n_samples}_{n_params}params/resampled_all_pressure_traces.csv')

# Select relevant inputs
relevant_columns = []
for col in df_x.columns:
    relevant_columns.append(col)
    if col == 'T':
        break

filtered_input = df_x[relevant_columns]
filtered_input = filtered_input.drop(boolean_index['0'].values)

print(f"shapes of input and output dataframes after boolean check:{filtered_input.shape}, {output.shape}")

# Adjust to closest valid Sobol sample size
M = filtered_input.shape[0] // (n_params + 2)  # Compute valid M
valid_N = M * (n_params + 2)  # Compute closest valid sample size

filtered_input = filtered_input.iloc[:valid_N].reset_index(drop=True)
output = output.iloc[:valid_N].reset_index(drop=True)

print(f"Using {valid_N} samples to match Sobol's structured sampling.")


print(f"Shapes of input and output dataframes using Valid {valid_N}: {filtered_input.shape}, {output.shape}")

# Define problem specification
problem = ProblemSpec({
    'num_vars': len(relevant_columns),
    'names': relevant_columns,
    'bounds': filtered_input[relevant_columns].describe().loc[['min', 'max']].T.values
})

# Set input samples
problem.set_samples(filtered_input)

# Store sensitivity results
sensitivity_results = {}

# Loop over each output column
for output_col in output.columns:
    print(f"Processing {output_col}...")

    # Set output values
    problem.set_results(output[output_col].values)

    print(problem.samples.shape, problem.results.shape)
    
    # Perform Sobol analysis
    sobol_indices = problem.analyze_sobol(calc_second_order=False, print_to_console=True)

    
    # Sort results
    total, first, = sobol_indices.to_df()
    total.sort_values('ST', inplace=True, ascending=False)

    # Save results
    result_data = pd.DataFrame({
        "Parameter": total.index,
        "ST": total['ST'],
        "ST_conf": total['ST_conf']
    })


    # Save results
    safe_name = output_col.replace("/", "_") # Avoid slashes in file names

    result_file = os.path.join(output_dir, f"sensitivity_{safe_name}.csv")
    result_data.to_csv(result_file, index=False)


print("Sensitivity analysis completed.")

In [16]:
output

,0,1,2,3,4,5,6,7,8,9,...,P_max,esp,sys,EF,dia,tau,Ees/Ea,iT,iHR,CO
0,2.419990,1.895984,1.473719,1.141043,0.884936,0.692407,0.551200,0.450288,0.380157,0.332907,...,41.825574,23.750153,24.957216,0.432162,0.182878,0.013462,0.761065,0.574517,104.435473,3.764469
1,6.620952,5.625962,4.819966,4.181106,3.685705,3.309951,3.031236,2.829099,2.685790,2.586486,...,57.186473,46.014429,47.112541,0.195362,2.401946,0.018443,0.242794,0.768634,78.060557,2.434115
2,7.262025,5.880079,4.760128,3.872522,3.184894,2.664501,2.280097,2.003290,1.809352,1.677564,...,78.902395,61.144071,62.809901,0.225067,1.328800,0.020908,0.290434,0.854233,70.238480,2.835045
3,3.142702,2.429830,1.855705,1.403485,1.055242,0.793178,0.600579,0.462461,0.365929,0.300301,...,55.085000,32.573303,37.981747,0.408672,0.138083,0.011098,0.691109,0.482692,124.302983,3.332913
4,3.881509,3.014830,2.314857,1.762155,1.335703,1.014375,0.778125,0.608831,0.490793,0.410924,...,67.648662,38.494934,42.632645,0.430958,0.155238,0.015083,0.757339,0.630132,95.218121,3.099857
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49987,3.964326,3.075231,2.354869,1.784090,1.342012,1.007522,0.760475,0.582569,0.457861,0.372994,...,66.678175,38.628091,40.386946,0.420679,0.140657,0.013473,0.726158,0.550316,109.028237,3.956697
49988,5.534149,4.387760,3.455180,2.713250,2.136244,1.697811,1.372561,1.137231,0.971418,0.857923,...,75.509741,48.643473,49.579170,0.355799,0.536371,0.019878,0.552310,0.779856,76.937267,3.029706
49989,2.855193,2.368005,1.993766,1.712604,1.506046,1.357687,1.253529,1.182067,1.134167,1.102812,...,53.730509,37.464871,39.311545,0.302726,1.052834,0.018504,0.434157,0.838589,71.548740,1.998822
49990,4.214710,3.313265,2.580809,1.998930,1.547214,1.204766,0.951462,0.768870,0.640827,0.553715,...,60.062660,36.836835,40.392426,0.386693,0.259692,0.012808,0.630505,0.502472,119.409749,4.143181
